In [2]:
# Load modules for data manipulation
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os

In [3]:
# Load modules for machine learning
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Calculate the accurancy of the model
from sklearn.metrics import accuracy_score, classification_report

In [4]:
load_dotenv()

True

In [5]:
eco = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'eco-products.csv', encoding='ISO-8859-1', sep='\t')
clt = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ38.csv', encoding='ISO-8859-1', sep='\t')
ctr = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ7S.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)

## Get the best ecological product

In [6]:
eco = [str(i).zfill(5) for i in eco['COPRO']]

In [7]:
## ECO: 00003 | 242568 - LIVRET DEVELOPPEMENT DURABLE ET SOLIDAIRE
## DEPOTS MONETAIRES - EPARGNE MONETAIRE LIQUIDE - LIVRETS REGLEMENTES
ctr[ctr['COPRO'].isin(eco)]['COPRO'].value_counts()

00003    242568
00546      5141
00548      4785
07649      2724
00940      2166
00509      1876
00547      1027
00941       388
03992       101
00565        83
06458        75
Name: COPRO, dtype: int64

## Concatinate with other tables TJ39 - TJDR - TJER

In [8]:
tj39 = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj39.csv', encoding='ISO-8859-1', sep=';')
tjdr = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjdr.csv', encoding='ISO-8859-1', sep=';')
tjer = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjer.csv', encoding='ISO-8859-1', sep=';')

In [9]:
tj39 = tj39[['COMAX', 'MSMENC', 'MTPATR', 'MTVAOP']]
tjdr = tjdr[['COMAX', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI']]
tjer = tjer[['COMAX', 'MTAPJE', 'MTCPJE', 'MTPJE']]

In [10]:
tj39['COMAX'] = [val[:10] for val in tj39['COMAX'].values]

In [11]:
# pour le cas de la table tj39, la variable COMAX elle est de 13 caracteres
# or celle des autres tables sont de 10, donc il fallait convertir de 13 a 10
# on prend les 10 premiers caractères du COMAX de tj39, d'apres le data engineer
# de la PBS.
clt[clt['COMAX'] == '1ce15b6b35']

,COMAX,AGE,COSEXE,CTSCPI,CESITC,DDVALE,DFVALE,COPOST,LAT,LON
517291,1ce15b6b35,59,1,2300,1,2013-10-21,9999-01-01,66330,42.6813,2.9412


In [12]:
# This table is only good fot the moral clients only
data = pd.merge(clt,tj39,on='COMAX',how='left')
data = pd.merge(data,tjer,on='COMAX',how='left')
data = pd.merge(data,tjdr,on='COMAX',how='left')

In [13]:
data = data.dropna()

In [14]:
data = data.drop_duplicates(subset=['COMAX'])

In [15]:
data.shape

(2747, 20)

## les avec le 00003

In [17]:
cols = ['AGE', 'COSEXE', 'CTSCPI', 'LAT', 'LON', 'MSMENC', 'MTPATR',
        'MTVAOP', 'MTAPJE', 'MTCPJE', 'MTPJE', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI']

In [18]:
def done(ecos, cols):
    print('THIS IS PRODUCT:', ecos)
    
    # get the ids of the products
    ids = ctr[ctr['COPRO'].isin(ecos)]['COMAX'].values
    data['ECO'] = data['COMAX'].isin(ids)
    
    # change the types
    data['CTSCPI'] = data['CTSCPI'].astype(str)
    data['COPOST'] = data['COPOST'].astype(str)
    
    ''' The Model'''
    
    # Labels are the values we want to predict
    labels = np.array(data['ECO'])

    # Remove the labels from the features
    features = np.array(data[cols])

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

    # Train the model on training data
    rf.fit(train_features, train_labels)

    # Use the forest's predict method on the test data
    predictions = rf.predict(test_features)

    # Classification report
    print(classification_report(test_labels, predictions.round()))    

In [20]:
for ecos in eco:
    done([ecos], cols)

THIS IS PRODUCT: ['00568']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['05808']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00943']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00942']
              precision    recall  f1-score   support

       False      

/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       535
        True       0.00      0.00      0.00        15

    accuracy                           0.97       550
   macro avg       0.49      0.50      0.49       550
weighted avg       0.95      0.97      0.96       550

THIS IS PRODUCT: ['00547']


/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

              precision    recall  f1-score   support

       False       0.99      1.00      1.00       547
        True       0.00      0.00      0.00         3

    accuracy                           0.99       550
   macro avg       0.50      0.50      0.50       550
weighted avg       0.99      0.99      0.99       550

THIS IS PRODUCT: ['03992']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00940']


...

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       549
        True       0.00      0.00      0.00         1

    accuracy                           1.00       550
   macro avg       0.50      0.50      0.50       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['07606']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00003']
              precision    recall  f1-score   support

       False       0.46      0.22      0.30       159
        True       0.74      0.90      0.81       391

    accuracy                           0.70       550
   macro avg       0.60      0.56      0.55       550
weighted avg       0.66      0.70      0.66       550

THIS IS PRODUCT: ['

/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

              precision    recall  f1-score   support

       False       0.98      1.00      0.99       537
        True       0.00      0.00      0.00        13

    accuracy                           0.98       550
   macro avg       0.49      0.50      0.49       550
weighted avg       0.95      0.98      0.96       550

THIS IS PRODUCT: ['06458']


/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/trinity/shared/apps/local/Python/3.6.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       550
         1.0       0.00      0.00      0.00         0

    accuracy                           1.00       550
   macro avg       0.50      0.50      0.50       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00509']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550

THIS IS PRODUCT: ['00565']
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       550

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550



In [21]:
done(eco, cols)

THIS IS PRODUCT: ['00568', '05808', '00943', '00942', '07648', '07649', '00546', '00547', '03992', '00940', '00941', '07606', '00003', '05807', '00548', '06458', '00509', '00565']
              precision    recall  f1-score   support

       False       0.42      0.15      0.22       148
        True       0.75      0.92      0.83       402

    accuracy                           0.71       550
   macro avg       0.58      0.54      0.52       550
weighted avg       0.66      0.71      0.66       550

